# Librerías

In [31]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import pyarrow.parquet as pq
import ast
from dateutil import parser
import re

# Endpoints

In [18]:
# Se cargan los dataframes de los tres archivos nuevamente

# steam_games
df_steam_games = pd.read_csv('/Users/mlucchesi/Henry/PI/data/csv/procesados/steam_games_procesado.csv')

# user_reviews
df_user_reviews = pd.read_csv('/Users/mlucchesi/Henry/PI/data/csv/procesados/user_reviews_procesado_nlp.csv')

# users_items
df_users_items = pd.read_csv('/Users/mlucchesi/Henry/PI/data/csv/procesados/users_items_procesado.csv')

### Archivo ```steam_games```

In [34]:
# Se crea una función que extraiga el año del lanzamiento del juego de 'release_date'

def get_year(date):
    try:
        date_object = parser.parse(date)
        return date_object.year
    except:
        return None
    
# Creo una columna con el año, usando la función creada

df_steam_games['year_developed'] = df_steam_games['release_date'].apply(get_year).astype('Int64')

In [44]:
# Muestro como quedo el dataframe
df_steam_games.head(1)

,Unnamed: 0,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,year_developed
0,88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,0.0,761140.0,Kotoshiro,2018


In [23]:
# Exporto a CSV el dataframe completo

df_steam_games.to_csv('/Users/mlucchesi/Henry/PI/data/csv/procesados/steam_games_year.csv', index=False)

### Archivo ```user_reviews```

In [45]:
# Creo una función que extraiga el año de la fecha de posteo de la review

def get_year_reviews(posted):
    match = re.search(r'\d{4}', posted)  
    if match:
        return match.group()
    else:
        return None 

df_user_reviews['year_review'] = df_user_reviews['posted'].apply(get_year_reviews)

In [46]:
# Muestro como quedó

df_user_reviews.head()

,Unnamed: 0,user_id,user_url,posted,item_id,helpful,recommend,sentiment_analysis,year_review
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,No ratings yet,True,2,2011
1,1,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,2,2014
2,2,evcentric,http://steamcommunity.com/id/evcentric,Posted February 3.,248820,No ratings yet,True,2,None
3,3,doctr,http://steamcommunity.com/id/doctr,"Posted October 14, 2013.",250320,2 of 2 people (100%) found this review helpful,True,2,2013
4,4,maplemage,http://steamcommunity.com/id/maplemage,"Posted April 15, 2014.",211420,35 of 43 people (81%) found this review helpful,True,1,2014


In [96]:
print(df_user_reviews['recommend'].unique())

[ True]


## Funciones

### ```UsersWorstDeveloper```
_Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)_

In [48]:
# Para optimizar, se extraen solo las columnas necesarias para hacer el merge

# Primero, se obtiene el developer y su respectivo 'id'

df_developer_id = df_steam_games[['developer', 'id']]
df_developer_id.head()

,developer,id
0,Kotoshiro,761140.0
1,Secret Level SRL,643980.0
2,Poolians.com,670290.0
3,彼岸领域,767400.0
4,NaN,773570.0


In [49]:
# Luego, se obtiene el 'id' de juego, el año de posteo de la review, y aquellos que fueron negativos (recommend = False y sentiment analysis = 0)

df_reviews_negativas = df_user_reviews[['item_id', 'recommend', 'sentiment_analysis', 'year_review']]
df_reviews_negativas.head()

,item_id,recommend,sentiment_analysis,year_review
0,1250,True,2,2011
1,251610,True,2,2014
2,248820,True,2,None
3,250320,True,2,2013
4,211420,True,1,2014


In [50]:
# Como se van a unir por id de juego, hay que renombrar las columnas para que coincidan

# En el df de los developers, se cambia 'id' por 'item_id'

df_developer_id = df_developer_id.rename(columns={'id': 'item_id'})
df_developer_id.head()

,developer,item_id
0,Kotoshiro,761140.0
1,Secret Level SRL,643980.0
2,Poolians.com,670290.0
3,彼岸领域,767400.0
4,NaN,773570.0


In [51]:
# Se mergean los dataframes

df_developer_negative_review = df_developer_id.merge(df_reviews_negativas, on='item_id', how='right')
df_developer_negative_review

,developer,item_id,recommend,sentiment_analysis,year_review
0,Tripwire Interactive,1250.0,True,2,2011
1,NaN,251610.0,True,2,2014
2,"Hopoo Games, LLC",248820.0,True,2,None
3,Telltale Games,250320.0,True,2,2013
4,FromSoftware,211420.0,True,1,2014
...,...,...,...,...,...
55827,Daybreak Game Company,218230.0,True,2,2015
55828,Forever Entertainment S. A.,294230.0,True,1,2015
55829,Nexon,273110.0,True,1,2015
55830,Evil Tortilla Games,427730.0,True,1,None


In [52]:
# Se exporta a un CSV

df_developer_negative_review.to_csv('/Users/mlucchesi/Henry/PI/data/csv/procesados/developer_negative_review.csv')

In [74]:
df_developer_negative_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55832 entries, 0 to 55831
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   developer           46772 non-null  object 
 1   item_id             55832 non-null  float64
 2   recommend           55832 non-null  bool   
 3   sentiment_analysis  55832 non-null  int64  
 4   year_review         55832 non-null  int64  
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 1.8+ MB


In [70]:
# Llena los valores None en la columna 'year_review' con 0
df_developer_negative_review['year_review'] = df_developer_negative_review['year_review'].fillna(0)

# Convierte la columna 'year_review' a tipo de datos int
df_developer_negative_review['year_review'] = df_developer_negative_review['year_review'].astype(int)

In [105]:
# Se define la función

def UsersWorstDeveloper(year: int):
    df_filtrado = df_developer_negative_review[(df_developer_negative_review['year_review'] == year) &
                                               (df_developer_negative_review['sentiment_analysis'] == 0)]

    if df_filtrado.empty:
        return None

    peores_devs = df_filtrado.groupby('developer')['recommend'].sum().sort_values(ascending=False).head(3)

    result = [{"Top {}".format(i + 1): developer} for i, (developer, _) in enumerate(peores_devs.items())]

    return result

In [118]:
# Probamos la función

UsersWorstDeveloper(2014)

[{'Top 1': 'Valve'},
 {'Top 2': 'Facepunch Studios'},
 {'Top 3': 'Bohemia Interactive'}]

### ```sentiment_analysis```
_Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor._

In [119]:
# Para optimizar, se extraen solo las columnas necesarias para hacer el merge

# Primero, se obtiene el id del juego con 'sentiment_analysis'

df_id_sentiment = df_user_reviews [['item_id', 'sentiment_analysis']]
df_id_sentiment.head()

,item_id,sentiment_analysis
0,1250,2
1,251610,2
2,248820,2
3,250320,2
4,211420,1


In [121]:
# Lo mismo, para el dataframe de steam_games

df_id_developer = df_steam_games[['id', 'developer']]
df_id_developer.head()

,id,developer
0,761140.0,Kotoshiro
1,643980.0,Secret Level SRL
2,670290.0,Poolians.com
3,767400.0,彼岸领域
4,773570.0,NaN


In [122]:
# Como se van a unir por id de juego, hay que renombrar las columnas para que coincidan

# En el df de los developers, se cambia 'id' por 'item_id'

df_id_developer = df_id_developer.rename(columns={'id': 'item_id'})
df_id_developer.head()

,item_id,developer
0,761140.0,Kotoshiro
1,643980.0,Secret Level SRL
2,670290.0,Poolians.com
3,767400.0,彼岸领域
4,773570.0,NaN


In [123]:
# Se mergean los dataframes

df_id_developer_sentiment = df_id_developer.merge(df_id_sentiment, on='item_id', how='right')
df_id_developer_sentiment.head()

,item_id,developer,sentiment_analysis
0,1250.0,Tripwire Interactive,2
1,251610.0,NaN,2
2,248820.0,"Hopoo Games, LLC",2
3,250320.0,Telltale Games,2
4,211420.0,FromSoftware,1


In [124]:
# Se define la función

def sentiment_analysis(developer: str):
    df_filtrado = df_id_developer_sentiment[df_id_developer_sentiment['developer'] == developer] 

    if df_filtrado.empty:
        return None
    
    count_sentiment_analysis = df_filtrado.groupby('sentiment_analysis').size().to_dict()

    resultado = {developer: [f'Negative = {count_sentiment_analysis.get(0, 0)}', f'Positive = {count_sentiment_analysis.get(2, 0)}']}

    return resultado


In [127]:
# Pruebo la función

sentiment_analysis('Valve')

{'Valve': ['Negative = 1543', 'Positive = 5120']}

### Exportar los dataframes

```df_id_developer_sentiment```

In [128]:
# Exporto el CSV

df_id_developer_sentiment.to_csv('/Users/mlucchesi/Henry/PI/data/csv/procesados/id_dev_sentiment.csv', index=False)

In [130]:
# Exporto en parquet, para que la API esté optimizada

df_id_developer_sentiment.to_parquet('/Users/mlucchesi/Henry/PI/MLOps/project/PI-MLOps/archivos/id_dev_sentiment.parquet', engine='pyarrow')

```df_developer_negative_review```

In [131]:
# Exporto el CSV

df_developer_negative_review.to_csv('/Users/mlucchesi/Henry/PI/data/csv/procesados/developer_negative_review.csv', index=False)

In [132]:
# Exporto en parquet, para que la API esté optimizada

df_developer_negative_review.to_parquet('/Users/mlucchesi/Henry/PI/MLOps/project/PI-MLOps/archivos/developer_negative_review.parquet', engine='pyarrow')